In [1]:
import string
import re, string
punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))
from collections import Counter
import numpy as np
from collections import defaultdict

In [2]:
with open("stopwords.txt", 'r') as r:
    stops = []
    for line in r:
        stops += [i.strip() for i in line.split('\t')]

In [3]:
import json
d = {}
with open("captions_train2014.json") as captions:
    d = json.load(captions)
import pickle
resnet = {}
with open("resnet18_features_train.pkl",mode="rb") as res:
    resnet = pickle.load(res)

In [4]:
d_ids = defaultdict(list, {})
d_ids[1].append(2)
d_ids[1].append(3)
print(d_ids)

defaultdict(<class 'list'>, {1: [2, 3]})


In [5]:
d_ids = d_ids = defaultdict(list, {})
for image in range(len(d["annotations"])):\
    d_ids[d["annotations"][image]["image_id"]].append(d["annotations"][image]["caption"])
happy_word_list = list(d_ids.values())

In [6]:
def strip_punc(corpus):
    """ Removes all punctuation from a string.

        Parameters
        ----------
        corpus : str

        Returns
        -------
        str
            the corpus with all punctuation removed"""
    # substitute all punctuation marks with ""
    return punc_regex.sub('', corpus)
def to_bag(arr,k=0,stop=[]):
    c = Counter()
    for a in arr:
        c += a
    for i in stop:
        del c[i]
    keys, other = unzip(c.most_common(k))
    return (keys,c)
def to_counter(s):
    return Counter(strip_punc(s.lower()).split())
def unzip(pairs):
    """Splits list of pairs (tuples) into separate lists.
    
    Example: pairs = [("a", 1), ("b", 2)] --> ["a", "b"] and [1, 2]
    
    This should look familiar from our review back at the beginning of week 1
    :)
    """
    return tuple(zip(*pairs))
def to_idf(arr,stop=[]):
    b,c = to_bag(arr,50,stop=stop)
    k, v = unzip(c.most_common())
    v = np.log10(len(arr)/(np.array(v)))
    return dict(zip(*(k,v)))
def captions_to_idf(arr):
    """
    Takes in an array of captions which are the captions and returns the big idf array
    """
    big_list_of_happiness = []
    for i in (range(len(arr))):
        for j in range(len(arr[i])):
            big_list_of_happiness.append(to_counter(arr[i][j]))
            if((5*i+j)%40000==0):
                print(len(big_list_of_happiness))
    return to_idf(big_list_of_happiness,stop=stops)
very_cool_array = captions_to_idf(happy_word_list)
print(len(very_cool_array))

1
40017
80035
120056
160081
200098
240118
280140
320161
360177
400195
24354


In [8]:
test = very_cool_array
print(test["happiness"])
print(test["dog"])

5.316088868597503
1.5160595093533684


In [10]:
print(len(very_cool_array))

24354


In [11]:
with open("IDF.pkl", mode="wb") as opened_file:
    pickle.dump(very_cool_array, opened_file)

In [13]:
with open("IDF.pkl", mode="rb") as opened_file:
    my_loaded_grades = pickle.load(opened_file)
my_loaded_grades["dog"]

1.5160595093533684